In [2]:
%pip install seaborn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import pandas as pd # working with tabular data
import pickle # for machine learning models
import seaborn as sns # visualization
import matplotlib.pyplot as plt # visualization

from sklearn.feature_extraction import DictVectorizer # Machine Learning
from sklearn.linear_model import LinearRegression # Machine Learning
from sklearn.linear_model import Lasso # Regularization
from sklearn.linear_model import Ridge # Regularization

from sklearn.metrics import mean_squared_error # Loss Function

In [4]:
import mlflow

# to hook up with MLFlow UI
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("mlops_nyc_taxi") # choose a name for your experiment

<Experiment: artifact_location=('file:///c:/Users/CPE '
 'KMUTT/Documents/git/CPE393-MLOps/MLExperimenttracking-cpe393/mlruns/1'), creation_time=1742459591224, experiment_id='1', last_update_time=1742459591224, lifecycle_stage='active', name='mlops_nyc_taxi', tags={}>

In [5]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-01.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [6]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
        df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [7]:
df_train = read_dataframe('yellow_tripdata_2022-01.parquet')
df_val = read_dataframe('yellow_tripdata_2022-02.parquet')

In [8]:
len(df_train), len(df_val)

(2421440, 2918187)

In [9]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [10]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [11]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [12]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred)

30.588143613519556

In [13]:
with open('./lin_reg.bin', 'wb') as f_out: # wb means write binary mlops-zoomcamp/week_1/models
    try:
        # Pickle both the dictionary vectorizer and the linear regression model
        pickle.dump((dv, lr), f_out)
        print("Model successfully pickled.")
    except Exception as e:
        print("Error occurred while pickling the model:", str(e))

Model successfully pickled.


In [14]:
%pip install xgboost
%pip install hyperopt

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [22]:
# import required modules
import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials # some methods to optimize hyperparameters
from hyperopt.pyll import scope

In [23]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [24]:
# Objective function - set the parameters for this specific run
def objective(params): 
    
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params) 
        booster = xgb.train(
            params=params,
            dtrain=train, # model trained on training set
            num_boost_round=3, # restricted due to time constraints - a value of 1000 iterations is common
            evals=[(valid, 'validation')], # model evaluated on validation set
            early_stopping_rounds=3 # if no improvements after 3 iterations, stop running # restricted, time constraints
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [31]:
# Set the range of the hyperparameter optimization search
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)), # tree depth 4 to 100. Returns float, so convert to integer
    'learning_rate': hp.loguniform('learning_rate', -3, 0), # range exp(-3), exp(0) which is (0.049787, 1.0)
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1), # range exp(-5), exp(-1) which is (0.006738, 0.367879)
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1), # range exp(-6), exp(-1) which is (0.002479, 0.367879)
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3), # range exp(-1), exp(3) which is (0.367879, 20.085537)
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin( # imported above
    fn=objective,
    space=search_space, # as defined above
    algo=tpe.suggest, # tpe is the algorithm used for optimization
    max_evals=3, #restricted, time constraints
    trials=Trials()
)
with open("best_result.pkl", "wb") as f_out:
  pickle.dump(best_result, f_out)

with open("dict_vectorizer.pkl", "wb") as f_out:
  pickle.dump(dv, f_out)

mlflow.log_artifact("best_result.pkl", artifact_path="best_result")
mlflow.log_artifact("dict_vectorizer.pkl", artifact_path="preprocessor")


  0%|          | 0/3 [00:00<?, ?trial/s, best loss=?]

c:\Users\CPE KMUTT\Documents\git\CPE393-MLOps\MLExperimenttracking-cpe393\.venv\Lib\site-packages\xgboost\callback.py:386: UserWarning: [23:30:03] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\objective\regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:9.05973                          
[1]	validation-rmse:8.49393                          
[2]	validation-rmse:8.00183                          
 33%|███▎      | 1/3 [06:06<12:13, 366.67s/trial, best loss: 64.02932110516502]

c:\Users\CPE KMUTT\Documents\git\CPE393-MLOps\MLExperimenttracking-cpe393\.venv\Lib\site-packages\xgboost\callback.py:386: UserWarning: [23:36:10] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\objective\regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:5.49354                                                    
[1]	validation-rmse:5.14328                                                    
[2]	validation-rmse:5.09308                                                    
 67%|██████▋   | 2/3 [06:56<03:00, 180.32s/trial, best loss: 25.939493342169417]

c:\Users\CPE KMUTT\Documents\git\CPE393-MLOps\MLExperimenttracking-cpe393\.venv\Lib\site-packages\xgboost\callback.py:386: UserWarning: [23:37:00] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\objective\regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



 67%|██████▋   | 2/3 [17:08<08:34, 514.14s/trial, best loss: 25.939493342169417]


KeyboardInterrupt: 

In [33]:
mlflow.end_run()

In [19]:
mlflow.xgboost.autolog(disable=True) # MLflow will not store parameters automatically - these will have to be requested

In [20]:
#best params

params = {
    'learning_rate': 0.7529276671138044,
    'max_depth': 23,
    'min_child_weight': 1.0231817552469362,
    'objective': 'reg:linear',
    'reg_alpha': 0.012244667129505497,
    'reg_lambda': 0.0054821155558349025,
    'seed': 42   
}

In [34]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.704653447185351,
        'max_depth': 81,
        'min_child_weight': 17.728749369797733,
        'objective': 'reg:linear',
        'reg_alpha': 0.28551023451325785,
        'reg_lambda': 0.1080566220694057,
        'seed': 42   
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=params,
        dtrain=train, # model trained on training set
        num_boost_round=3, # restricted due to time constraints - a value of 1000 iterations is common
        evals=[(valid, 'validation')], # model evaluated on validation set
        early_stopping_rounds=3 # if no improvements after 3 iterations, stop running # restricted, time constraints
        )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    with open("preprocessor.b", "wb") as f_out: # save pre-processing as a model
        pickle.dump(dv, f_out)
        
    mlflow.log_artifact("preprocessor.b", artifact_path="preprocessor") # we can isolate the pre-processing from raw data
    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow") 

c:\Users\CPE KMUTT\Documents\git\CPE393-MLOps\MLExperimenttracking-cpe393\.venv\Lib\site-packages\xgboost\callback.py:386: UserWarning: [23:49:57] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\objective\regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()


[0]	validation-rmse:5.78986
[1]	validation-rmse:5.29172
[2]	validation-rmse:5.20855


c:\Users\CPE KMUTT\Documents\git\CPE393-MLOps\MLExperimenttracking-cpe393\.venv\Lib\site-packages\mlflow\xgboost\__init__.py:168: UserWarning: [23:50:11] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  xgb_model.save_model(model_data_path)
2025/03/27 23:50:20 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [37]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.8225124825252088,
        'max_depth': 54,
        'min_child_weight': 1.513023703610907,
        'objective': 'reg:linear',
        'reg_alpha': 0.285846971105999,
        'reg_lambda': 0.012685569323142435,
        'seed': 42   
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train, # model trained on training set
        num_boost_round=3, # restricted due to time constraints - a value of 1000 iterations is common
        evals=[(valid, 'validation')], # model evaluated on validation set
        early_stopping_rounds=3 # if no improvements after 3 iterations, stop running # restricted, time constraints
        )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    with open("preprocessor.b", "wb") as f_out: # save pre-processing as a model
        pickle.dump(dv, f_out)
        
    mlflow.log_artifact("preprocessor.b", artifact_path="preprocessor") # we can isolate the pre-processing from raw data
    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow") 

c:\Users\CPE KMUTT\Documents\git\CPE393-MLOps\MLExperimenttracking-cpe393\.venv\Lib\site-packages\xgboost\callback.py:386: UserWarning: [23:55:19] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\objective\regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()


[0]	validation-rmse:5.49354
[1]	validation-rmse:5.14328
[2]	validation-rmse:5.09308


c:\Users\CPE KMUTT\Documents\git\CPE393-MLOps\MLExperimenttracking-cpe393\.venv\Lib\site-packages\mlflow\xgboost\__init__.py:168: UserWarning: [23:56:13] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  xgb_model.save_model(model_data_path)
2025/03/27 23:56:21 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [ ]:
logged_model = './mlruns/1/runs/35b95801a40d413fa0091d6ceb6ae0c7/artifacts/models_mlflow'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

NameError: name 'mlflow' is not defined

In [ ]:
loaded_model

NameError: name 'loaded_model' is not defined

In [ ]:
xgboost_model = mlflow.xgboost.load_model(logged_model)

OSError: No such file or directory: '.\mlruns\1\runs\35b95801a40d413fa0091d6ceb6ae0c7\artifacts\models_mlflow'

In [ ]:
xgboost_model

NameError: name 'xgboost_model' is not defined

In [ ]:
y_pred = xgboost_model.predict(valid)

NameError: name 'xgboost_model' is not defined

In [ ]:
# check the first 10
y_pred[:10]

array([14.876949, 19.59639 , 30.903755, 26.593918, 30.903755, 13.210818,
       24.073948,  4.972286, 17.644482, 16.71025 ], dtype=float32)

Model Registry